# Data Preprocessing

In [1]:
from jamo import hangul_to_jamo
import pandas as pd
import numpy as np
import os, glob, librosa, re, scipy
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
import torch

# Hyperparameters for Generating Mel-spectrogram 
sample_rate = 22050
preemphasis = 0.97
n_fft = 1024
hop_length = 256
win_length = 1024
ref_db = 20
max_db = 100
mel_dim = 80

# Major Hyperparameters
batch_size = 8

In [2]:
from jamo import hangul_to_jamo

PAD = '_'
EOS = '~'
SPACE = ' '

JAMO_LEADS = "".join([chr(_) for _ in range(0x1100, 0x1113)])
JAMO_VOWELS = "".join([chr(_) for _ in range(0x1161, 0x1176)])
JAMO_TAILS = "".join([chr(_) for _ in range(0x11A8, 0x11C3)])
ETC = ".!?"

VALID_CHARS = JAMO_LEADS + JAMO_VOWELS + JAMO_TAILS + SPACE + ETC
symbols = PAD + EOS + VALID_CHARS

_symbol_to_id = {s: i for i, s in enumerate(symbols)}
_id_to_symbol = {i: s for i, s in enumerate(symbols)}

# text를 초성, 중성, 종성으로 분리하여 id로 반환하는 함수
def text_to_sequence(text):
    sequence = []
    if not 0x1100 <= ord(text[0]) <= 0x1113:
        text = ''.join(list(hangul_to_jamo(text)))
    for s in text:
        sequence.append(_symbol_to_id[s])
    sequence.append(_symbol_to_id['~'])
    return sequence

def sequence_to_text(sequence):
    result = ''
    for symbol_id in sequence:
        if symbol_id in _id_to_symbol:
            s = _id_to_symbol[symbol_id]
            result += s
    return result.replace('}{', ' ')

In [7]:
data_dir = "C:/Users/Poco/Jupyter/PaperReview/dataset/filelists-20200801.zip"
text_dir = data_dir + "/ljs_audio_text_train_filelist.txt"
filters = '([,])'

metadata = pd.read_csv(text_dir, dtype='object', sep='|', header=None)
text = metadata[1].values

out_dir = data_dir + '/data'
os.makedirs(out_dir, exist_ok=True)
os.makedirs(out_dir + '/text', exist_ok=True)
os.makedirs(out_dir + '/wav', exist_ok=True)
os.makedirs(out_dir + '/mel', exist_ok=True)

# text
print('Load Text')
text_len = []
for idx, s in enumerate(tqdm(text)):
    # 문자열에서 정규표현식을 이용하여 특정 문자열을 필터링하고,
    # 이를 빈 문자열('')로 대체한다.
    sentence = re.sub(re.compile(filters), '', s)
    sentence = text_to_sequence(sentence)
    
    text_len.append(len(sentence))
    text_name = 'sce-text-%05d.npy' % idx
    np.save(os.path.join(out_dir + '/text', text_name), sentence, allow_pickle=False)
print('Text Done')

Load Text


100%|██████████████████████████████████████████████████████████| 3076/3076 [00:01<00:00, 1645.87it/s]

Text Done


In [5]:
# HiFi-GAN 저자 구현
def mel_spectrogram(y, n_fft=1024, num_mels=80, sampling_rate=22050, hop_size=256, win_size=1024, fmin=0, fmax=8000, center=False):
    """
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))
    """

    mel = librosa.filters.mel(sampling_rate, n_fft, num_mels, fmin, fmax)
    
    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=torch.hann_window(win_size).to(y.device),
                      center=center, pad_mode='reflect', normalized=False, onesided=True)

    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9))

    spec = torch.matmul(torch.from_numpy(mel).float().to(y.device), spec)
    spec = torch.log(torch.clamp(spec, min=1e-5) * 1)

    return spec

In [11]:
# audio
wav_dir = metadata[0].values

print('Load Audio')
for idx, fn in enumerate(tqdm(wav_dir)):
    file_dir = data_dir + "/" +fn
    wav, _ = librosa.load(file_dir, sr=sample_rate)
    wav, _ = librosa.effects.trim(wav) # 묵음 제거
    
    mel = mel_spectrogram(torch.tensor(wav).unsqueeze(0)) # (1, 80, F)
    mel = mel.squeeze(0).T # (F, 80)
    mel = mel.numpy()

    mel_name = 'sce-mel-%05d.npy' % idx
    np.save(os.path.join(out_dir + '/mel', mel_name), mel, allow_pickle=False)
    
    # Wavenet vocoder에서 이용할 wav 파일을 추가로 저장함.
    wav_name = 'sce-wav-%05d.npy' % idx
    np.save(os.path.join(out_dir + '/wav', wav_name), wav, allow_pickle=False)

print('Audio Done')

Load Audio


  0%|                                                                       | 0/3076 [00:00<?, ?it/s]C:\Users\Poco\anaconda3\lib\site-packages\torch\functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:804.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
100%|███████████████████████████████████████████████████████████| 3076/3076 [00:20<00:00, 148.76it/s]

Audio Done
